In [5]:
import qiskit
import qiskit_machine_learning as qml
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit import Parameter, ParameterExpression, ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels.parameterized_quantum_kernel import ParamQuantumKernel

from qiskit import QuantumCircuit, Aer
from qiskit.aqua import QuantumInstance

from functools import partial,reduce
import numpy as np
import copy
from copy import deepcopy as dc

# Generate a trivial dataset

In [6]:
x_vec = [[ 0,  2],
         [.1, .4],
         [.5, .3],
         [.8, .2]]
y = [1, 1, -1, -1]

### Define the objective function used to train the PQK

In [7]:
def obj_align(params, PQK=None, x_vec=x_vec, y=y):
    # check that params are the right dimension
    PQK.bind_free_params(params)
    K = PQK.evaluate(x_vec)
    y = np.array(y)
    return y.T @ K @ y

optimizer = SPSA()

### Instantiate and train the PQK

In [8]:
# Define my feature map, partition my parameters
n_qubits = 4
fm = ZZFeatureMap(n_qubits, reps=2)
input_params = fm.parameters[::2]
free_params = fm.parameters[1::2]

# instantiate a simulator backend
qi = QuantumInstance(backend=Aer.get_backend('qasm_simulator'))

# Define my parameterized Kernel
pqk = ParamQuantumKernel(fm, free_params, input_params, quantum_instance=qi)

# Bind neccesary inputs to objective_function, such that it
# has `f(free_param_values) = real-scalar` usage.
alignment = partial(obj_align, PQK=pqk, x_vec=x_vec, y=y)

# instantiate an optimizer
spsa = SPSA(maxiter = 50)

# do kernel alignment
results = pqk.train_kernel(objective_function=alignment, 
                           optimizer=spsa)


In [16]:
results.__dict__

{'_optimizer_evals': 100,
 '_optimizer_time': 66.47269201278687,
 '_optimal_value': 3.2460937499999982,
 '_optimal_point': array([ 2.74991989, -1.86027882]),
 '_optimal_parameters': {ParameterVectorElement(x[0]): 2.7499198917550705,
  ParameterVectorElement(x[2]): -1.8602788172539544}}

In [36]:
pqk.evaluate(x_vec)

array([[1.        , 0.00585938, 0.18945312, 0.06054688],
       [0.00585938, 1.        , 0.05371094, 0.04394531],
       [0.18945312, 0.05371094, 1.        , 0.10742188],
       [0.06054688, 0.04394531, 0.10742188, 1.        ]])

# There's still bugs downstream when we try to use the final kernel

In [35]:
from qiskit_machine_learning.algorithms import QSVC
qsvc = QSVC(quantum_kernel = pqk)
qsvc.fit(x_vec, y)

/Users/brycefuller/anaconda3/lib/python3.7/site-packages/qiskit/providers/aer/aerjob.py:56: DeprecationWarning: The BaseJob abstract interface is deprecated as of the 0.18.0 release and will be removed in a future release. Instead you should build your backends using the JobV1 abstract class (which is the current latest version of the backend interface)
  super().__init__(backend, job_id)


RuntimeError: scikit-learn estimators should always specify their parameters in the signature of their __init__ (no varargs). <class 'qiskit_machine_learning.algorithms.classifiers.qsvc.QSVC'> with constructor (self, *args, quantum_kernel: Union[qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel, NoneType] = None, **kwargs) doesn't  follow this convention.